# Validation - distribution of GT and optimized AP potentials

This notebook evaluates qualitatively the distributions of membrane potentials and transmembrane currents of an action potential over the neuron morphology.

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import sys
import shutil

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import matplotlib.pyplot as plt
from scipy.spatial import distance
import MEAutility as mu
import json
import time
import numpy as np
from pathlib import Path

import pandas as pd
import seaborn as sns
from scipy.spatial.distance import cosine

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
base_path = Path("../..")

In [ ]:
colors_dict = {"soma": "C0",
               "all": "C1",
               "sections": "C2",
               "single": "C3"}
figsize = (10, 7)

## Load results and define model

In [ ]:
probe_type = "planar"
model_name = "hay_ais" # "hay", "hay_ais", "hay_ais_hillock"
cell_models_folder = base_path / "cell_models"

model_folder = cell_models_folder / f"{model_name}"

In [ ]:
# change this with folder containing your pkl file
results_date = '211124'  # '211124' '220111' # 
result_folder = base_dir / "results" / results_date

In [ ]:
pkl_file_name = "runs.pkl"

data = pickle.load(open(result_folder / pkl_file_name, 'rb'))
df_optimization = pd.DataFrame(data)
df_model = df_optimization.query(f"model == '{model_name}'")

In [ ]:
# set strategy column
df_model.loc[:, "strategy"] = df_model["extra_strategy"].values.copy()
df_model.loc[df_model["feature_set"] == "soma", "strategy"] = "soma"

In [ ]:
protocols_file = model_folder / "fitting" / "efeatures" / "protocols_BPO_all.json"
features_file = model_folder / "fitting" / "efeatures" / "features_BPO_all.json"

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_name, release=False)
cell_release = mf.create_ground_truth_model(model_name=model_name, release=True)

probe = mf.define_electrode(probe_type=probe_type)

param_names = [param.name for param in cell.params.values() if not param.frozen]

params_release = {}
for param in cell_release.params_by_names(param_names):
    params_release[param.name] = param.value

In [ ]:
protocol_for_eap = "IDrest_300"

### Define more recording points 

In [ ]:
extra_kwargs = mf.utils.get_extra_kwargs()

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    **extra_kwargs
)

In [ ]:
mf.plot_cell(eva_extra.cell_model, eva_extra.sim, param_values=params_release)

In [ ]:
positions = np.array([[-62, 828], [-3, 954], [-27, 546], [-27, 85], [134, -28], [-117, -189]])
position_names = ["apical_distal_left", "apical_distal_right", "apical_middle", "apical_proximal", 
                  "basal_right", "basal_left"]

In [ ]:
extra_recordings = mf.utils.extra_recordings_from_positions(cell_release, eva_extra.sim, positions, position_names)

In [ ]:
extra_recordings

In [ ]:
eva_extra = mf.create_evaluator(
    model_name=model_name,
    feature_set="extra",
    extra_strategy="all",
    protocols_with_lfp=protocol_for_eap,
    extra_recordings=dict(IDrest_300=extra_recordings), 
    **extra_kwargs
)

# Load protocols and original features

In [ ]:
idrest = eva_extra.fitness_protocols["IDrest_300"]

In [ ]:
opt_results = {}
for strategy in np.unique(df_model.strategy):
    opt_results[strategy] = {}
    opt_df = df_model.query(f"strategy == '{strategy}'")
    best_idx = np.argmin(opt_df.best_fitness)
    params_sample = opt_df.iloc[best_idx]
    params_dict = {k: v for k, v in zip(param_names, params_sample.best_params)}
    opt_results[strategy]["best_fitness"] = params_sample.best_fitness
    opt_results[strategy]["best_params"] = params_dict
    print(f"{strategy} --  best fitness: {params_sample.best_fitness}")

In [ ]:
ms_after = 50
ms_before = 10

In [ ]:
print("Computing RELEASE")
responses_release = eva_extra.run_protocol(idrest, params_release)
responses_cut_release = mf.utils.get_peak_cutout(responses_release, ms_before=ms_before, 
                                                 ms_after=ms_after, average=True)

for strategy in np.unique(df_model.strategy):
    print(f"Computing {strategy}")
    best_params = opt_results[strategy]["best_params"]

    responses = eva_extra.run_protocol(idrest, best_params)
    opt_results[strategy]["responses"] = responses
    responses_cut = mf.utils.get_peak_cutout(responses, ms_before=ms_before, 
                                             ms_after=ms_after, average=True)
    opt_results[strategy]["responses_cut"] = responses_cut

In [ ]:
responses_to_plot = []
for resp in responses_release:
    if any(pos_name in resp for pos_name in position_names):
        responses_to_plot.append(resp)

In [ ]:
distance_arr = []
strategy_arr = []
position_arr = []
for resp_name in responses_to_plot:
    fig, ax = plt.subplots(figsize=figsize)
    position = resp_name.split(".")[1]
    ax.plot(responses_cut_release[resp_name]["time"], responses_cut_release[resp_name]["voltage"], 
            color="k", label="GT")
    for strategy in opt_results:
        resp_cut = opt_results[strategy]["responses_cut"]
        ax.plot(resp_cut[resp_name]["time"], resp_cut[resp_name]["voltage"], 
                color=colors_dict[strategy], label=strategy.upper())
        dist = cosine(responses_cut_release[resp_name]["voltage"],
                      resp_cut[resp_name]["voltage"])
        position_arr.append(position)
        distance_arr.append(dist)
        strategy_arr.append(strategy)
    ax.set_xlabel("time (ms)", fontsize=12)
    ax.set_ylabel("$V_m$ (mV)", fontsize=12)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_title(f"{position}", fontsize=15)
    ax.axvline(ms_before, color="gray", ls="--")
    ax.legend()
    ax.axvline(ms_before, color="gray", ls="--")
    ax.legend()

df = pd.DataFrame({"strategy": strategy_arr, "distance": distance_arr, "position": position_arr})

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
sns.barplot(data=df, x="strategy", y="distance", hue="position", ax=ax)
ax.set_xlabel("Strategy", fontsize=12)
ax.set_ylabel("cos. dist.", fontsize=12)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_title(f"Response distances", fontsize=15)